In [4]:
import pathlib
from torch.utils.data import Dataset
import torch

In [5]:
class LatinDataset(Dataset):
    def __init__(self, path_dataset: pathlib.Path):
        self.path_dataset = path_dataset
        self.data_list = [x for x in self.path_dataset.glob('**/*') if x.is_file()]
        self.data_class = list(set(x.parent for x in self.data_list))
        

    def __len__(self):
        return len(self.data_list)
    
    def __getitem__(self, index):
        img_path = self.data_list[index]
        img_label = torch.zeroes(len(self.data_class))
        img_label[self.data_class.index(img_path.parent)] = 1.0